In [70]:
import FinanceDataReader as fdr
import os
import openpyxl
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
from pykrx import stock
import pandas_datareader.data as pdr
from datetime import datetime, timedelta

In [71]:
# Define the directory containing the Excel files
folder_path = './kospi/kospi_component'  # Adjust the path as needed

# Initialize an empty DataFrame to store the results
kospi = pd.DataFrame()

In [72]:
# def read_excel_files(folder_path):
#     # Create an empty list to store index codes
#     idx_code = list()
    
#     # Loop through all files in the specified folder
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".xlsx") or filename.endswith(".xls"):
#             file_path = os.path.join(folder_path, filename)
#             # Read the Excel file
#             df = pd.read_excel(file_path, dtype=str)
            
#             # Extract the '종목코드' column if it exists
#             if '종목코드' in df.columns:
#                 idx_code.extend(df['종목코드'].tolist())
    
#     # Remove duplicates by converting the list to a set
#     idx_code = set(idx_code)
    
#     # Convert the set back to a list and create a DataFrame
#     df_result = pd.DataFrame([list(idx_code)])
    
#     return df_result

In [73]:
# result_df = read_excel_files(folder_path)

In [74]:
# def get_unique_tickers(start_date, end_date, market="KOSPI"):
#     # Convert string dates to datetime objects
#     start = datetime.strptime(start_date, '%Y-%m-%d')
#     end = datetime.strptime(end_date, '%Y-%m-%d')
    
#     # Initialize an empty list to store unique tickers
#     unique_tickers = list()
    
#     # Iterate through each day in the date range
#     current_date = start
#     while current_date <= end:
#         # Get the list of tickers for the current date
#         tickers = stock.get_market_ticker_list(current_date.strftime('%Y-%m-%d'), market=market)
#         # Add tickers to the set (sets automatically handle duplicates)
#         unique_tickers.append(tickers)
#         # Move to the next day
#         current_date += timedelta(days=1)
    
#     unique_tickers = set(unique_tickers)
#     # Convert the set to a DataFrame
#     tickers_df = pd.DataFrame(list(unique_tickers))
#     tickers_df.columns = tickers_df.iloc[0]
#     tickers_df = tickers_df[1:].reset_index(drop=True)
#     return tickers_df

# # Define the date range
# start_date = '2006-01-01'
# end_date = '2022-12-31'

# # Get the DataFrame of unique tickers
# unique_tickers_df = get_unique_tickers(start_date, end_date)

# # Display the DataFrame
# unique_tickers_df.head()

In [75]:
# date_range = pd.date_range(start='2006-01-01', end='2022-12-31', freq='D')
# # DataFrame 생성
# date_df = pd.DataFrame(date_range, columns=['date'])
# date_df['^KS11'] = np.nan

In [76]:
# kospi = pd.concat([date_df, result_df], axis=1)
# kospi.head()

Yahho finance에서 데이터 가져오기- 수정주가

In [77]:
kospi = pd.read_csv('.\kospi\kospi_stock_data.csv', parse_dates=['date'], index_col='date')

In [78]:
unique_tickers = kospi.columns
unique_tickers

Index(['^KS11', '093050', '000040', '077970', '003550', '052690', '088790',
       '072130', '009070', '000320',
       ...
       '000990', '011000', '018260', '001570', '172580', '009410', '008040',
       '001970', '002630', '003220'],
      dtype='object', length=933)

In [79]:
kospi.head()

,^KS11,093050,000040,077970,003550,052690,088790,072130,009070,000320,...,000990,011000,018260,001570,172580,009410,008040,001970,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,1389.270020,NaN,5879.99707,126400.0,34390.574219,NaN,NaN,9000.0,1658.556030,5508.643555,...,16400.0,16608.988281,NaN,862.071350,NaN,4607.680176,3023.076904,NaN,4103.958984,2208.189209
2006-01-03,1394.869995,NaN,6119.99707,135200.0,33512.984375,NaN,NaN,9000.0,1632.539429,5427.033691,...,16200.0,16854.441406,NaN,866.706177,NaN,4554.102539,3038.461426,NaN,4247.016602,2202.324219
2006-01-04,1402.109985,NaN,5999.99707,123200.0,34829.367188,NaN,NaN,9420.0,1652.051880,5427.033691,...,16100.0,19349.878906,NaN,857.436584,NaN,4688.046387,3292.307617,NaN,4077.135742,2202.324219
2006-01-05,1395.510010,NaN,5999.99707,124800.0,35761.808594,NaN,NaN,9350.0,1652.051880,5488.241211,...,16800.0,19718.058594,NaN,829.627808,NaN,4554.102539,3292.307617,NaN,4050.312500,2199.391602
2006-01-06,1412.780029,NaN,5939.99707,126000.0,38778.527344,NaN,NaN,9450.0,1665.060181,5671.862305,...,17750.0,22377.132812,NaN,834.262634,NaN,5083.181641,3323.076904,NaN,4112.900391,2190.593994


In [80]:
def update_ticker_data(ticker, df, start_date='2006-01-01', end_date='2022-12-31'):
    # Append .KS to the ticker for yfinance
    ticker = str(ticker)
    ticker_ks = ticker + '.KS'
    try:
        # Fetch historical market data using yfinance
        ticker_data = yf.download(ticker_ks, start=start_date, end=end_date)
        ticker_data = ticker_data['Close']
        df[ticker] = ticker_data
    except Exception as e:
        print(f"Failed to fetch data for {ticker} using yfinance: {e}")
        try:
            # If yfinance fails, fetch data using pandas_datareader and naver
            ticker_data = pdr.DataReader(ticker, 'naver', start=start_date, end=end_date)
            ticker_data = ticker_data['Close']
            df[ticker] = ticker_data
        except Exception as e:
            print(f"Failed to fetch data for {ticker} using naver: {e}")
            return df
    return df

In [81]:
# Update the dataframe with the closing prices for each unique ticker
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    kospi = update_ticker_data(ticker, kospi)

Fetching data for ^KS11...


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['^KS11.KS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Fetching data for 093050...


[*********************100%%**********************]  1 of 1 completed


Fetching data for 000040...


...



1 Failed download:
['001970.KS']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)')


Fetching data for 002630...


[*********************100%%**********************]  1 of 1 completed


Fetching data for 003220...


[*********************100%%**********************]  1 of 1 completed


In [82]:
kospi.head()

,^KS11,093050,000040,077970,003550,052690,088790,072130,009070,000320,...,000990,011000,018260,001570,172580,009410,008040,001970,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,NaN,NaN,5879.99707,126400.0,34390.574219,NaN,NaN,9000.0,1658.556030,5508.643555,...,16400.0,16608.988281,NaN,862.071350,NaN,4607.680176,3023.076904,NaN,4103.958984,2208.189209
2006-01-03,NaN,NaN,6119.99707,135200.0,33512.984375,NaN,NaN,9000.0,1632.539429,5427.033691,...,16200.0,16854.441406,NaN,866.706177,NaN,4554.102539,3038.461426,NaN,4247.016602,2202.324219
2006-01-04,NaN,NaN,5999.99707,123200.0,34829.367188,NaN,NaN,9420.0,1652.051880,5427.033691,...,16100.0,19349.878906,NaN,857.436584,NaN,4688.046387,3292.307617,NaN,4077.135742,2202.324219
2006-01-05,NaN,NaN,5999.99707,124800.0,35761.808594,NaN,NaN,9350.0,1652.051880,5488.241211,...,16800.0,19718.058594,NaN,829.627808,NaN,4554.102539,3292.307617,NaN,4050.312500,2199.391602
2006-01-06,NaN,NaN,5939.99707,126000.0,38778.527344,NaN,NaN,9450.0,1665.060181,5671.862305,...,17750.0,22377.132812,NaN,834.262634,NaN,5083.181641,3323.076904,NaN,4112.900391,2190.593994


In [83]:
ticker_data = yf.download('^KS11', start='2006-01-01', end='2022-12-31')
ticker_data = ticker_data['Close']
kospi['^KS11'] = ticker_data
kospi.head()

[*********************100%%**********************]  1 of 1 completed


,^KS11,093050,000040,077970,003550,052690,088790,072130,009070,000320,...,000990,011000,018260,001570,172580,009410,008040,001970,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,1389.270020,NaN,5879.99707,126400.0,34390.574219,NaN,NaN,9000.0,1658.556030,5508.643555,...,16400.0,16608.988281,NaN,862.071350,NaN,4607.680176,3023.076904,NaN,4103.958984,2208.189209
2006-01-03,1394.869995,NaN,6119.99707,135200.0,33512.984375,NaN,NaN,9000.0,1632.539429,5427.033691,...,16200.0,16854.441406,NaN,866.706177,NaN,4554.102539,3038.461426,NaN,4247.016602,2202.324219
2006-01-04,1402.109985,NaN,5999.99707,123200.0,34829.367188,NaN,NaN,9420.0,1652.051880,5427.033691,...,16100.0,19349.878906,NaN,857.436584,NaN,4688.046387,3292.307617,NaN,4077.135742,2202.324219
2006-01-05,1395.510010,NaN,5999.99707,124800.0,35761.808594,NaN,NaN,9350.0,1652.051880,5488.241211,...,16800.0,19718.058594,NaN,829.627808,NaN,4554.102539,3292.307617,NaN,4050.312500,2199.391602
2006-01-06,1412.780029,NaN,5939.99707,126000.0,38778.527344,NaN,NaN,9450.0,1665.060181,5671.862305,...,17750.0,22377.132812,NaN,834.262634,NaN,5083.181641,3323.076904,NaN,4112.900391,2190.593994


In [84]:
# Save the updated dataframe to a CSV file
output_file_path = '.\kospi\kospi_stock_data.csv'
kospi.to_csv(output_file_path, index=True)

데이터 클랜징

In [85]:
kospi = pd.read_csv('.\kospi\kospi_stock_data.csv', parse_dates=['date'], index_col='date')
kospi.head()

,^KS11,093050,000040,077970,003550,052690,088790,072130,009070,000320,...,000990,011000,018260,001570,172580,009410,008040,001970,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,1389.270020,NaN,5879.99707,126400.0,34390.574219,NaN,NaN,9000.0,1658.556030,5508.643555,...,16400.0,16608.988281,NaN,862.071350,NaN,4607.680176,3023.076904,NaN,4103.958984,2208.189209
2006-01-03,1394.869995,NaN,6119.99707,135200.0,33512.984375,NaN,NaN,9000.0,1632.539429,5427.033691,...,16200.0,16854.441406,NaN,866.706177,NaN,4554.102539,3038.461426,NaN,4247.016602,2202.324219
2006-01-04,1402.109985,NaN,5999.99707,123200.0,34829.367188,NaN,NaN,9420.0,1652.051880,5427.033691,...,16100.0,19349.878906,NaN,857.436584,NaN,4688.046387,3292.307617,NaN,4077.135742,2202.324219
2006-01-05,1395.510010,NaN,5999.99707,124800.0,35761.808594,NaN,NaN,9350.0,1652.051880,5488.241211,...,16800.0,19718.058594,NaN,829.627808,NaN,4554.102539,3292.307617,NaN,4050.312500,2199.391602
2006-01-06,1412.780029,NaN,5939.99707,126000.0,38778.527344,NaN,NaN,9450.0,1665.060181,5671.862305,...,17750.0,22377.132812,NaN,834.262634,NaN,5083.181641,3323.076904,NaN,4112.900391,2190.593994


In [86]:
kospi['^KS11'].isna().sum()

14

In [87]:
#모든 행이 NA면 제거(공휴일)
kospi = kospi.dropna(how='all')
kospi.head()

,^KS11,093050,000040,077970,003550,052690,088790,072130,009070,000320,...,000990,011000,018260,001570,172580,009410,008040,001970,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,1389.270020,NaN,5879.99707,126400.0,34390.574219,NaN,NaN,9000.0,1658.556030,5508.643555,...,16400.0,16608.988281,NaN,862.071350,NaN,4607.680176,3023.076904,NaN,4103.958984,2208.189209
2006-01-03,1394.869995,NaN,6119.99707,135200.0,33512.984375,NaN,NaN,9000.0,1632.539429,5427.033691,...,16200.0,16854.441406,NaN,866.706177,NaN,4554.102539,3038.461426,NaN,4247.016602,2202.324219
2006-01-04,1402.109985,NaN,5999.99707,123200.0,34829.367188,NaN,NaN,9420.0,1652.051880,5427.033691,...,16100.0,19349.878906,NaN,857.436584,NaN,4688.046387,3292.307617,NaN,4077.135742,2202.324219
2006-01-05,1395.510010,NaN,5999.99707,124800.0,35761.808594,NaN,NaN,9350.0,1652.051880,5488.241211,...,16800.0,19718.058594,NaN,829.627808,NaN,4554.102539,3292.307617,NaN,4050.312500,2199.391602
2006-01-06,1412.780029,NaN,5939.99707,126000.0,38778.527344,NaN,NaN,9450.0,1665.060181,5671.862305,...,17750.0,22377.132812,NaN,834.262634,NaN,5083.181641,3323.076904,NaN,4112.900391,2190.593994


In [88]:
#모든 값이 Nan인 종목제거(해당기간에 없었음)
kospi = kospi.dropna(axis=1, how='all')
kospi.head()

,^KS11,093050,000040,077970,003550,088790,072130,009070,000320,023960,...,005980,001500,005180,000990,011000,001570,009410,008040,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,1389.270020,NaN,5879.99707,126400.0,34390.574219,NaN,9000.0,1658.556030,5508.643555,1540.254395,...,NaN,8971.159180,39250.0,16400.0,16608.988281,862.071350,4607.680176,3023.076904,4103.958984,2208.189209
2006-01-03,1394.869995,NaN,6119.99707,135200.0,33512.984375,NaN,9000.0,1632.539429,5427.033691,1543.735229,...,36750.035156,8743.079102,38700.0,16200.0,16854.441406,866.706177,4554.102539,3038.461426,4247.016602,2202.324219
2006-01-04,1402.109985,NaN,5999.99707,123200.0,34829.367188,NaN,9420.0,1652.051880,5427.033691,1561.139282,...,36750.035156,8438.971680,44500.0,16100.0,19349.878906,857.436584,4688.046387,3292.307617,4077.135742,2202.324219
2006-01-05,1395.510010,NaN,5999.99707,124800.0,35761.808594,NaN,9350.0,1652.051880,5488.241211,1482.821289,...,36450.035156,8438.971680,44500.0,16800.0,19718.058594,829.627808,4554.102539,3292.307617,4050.312500,2199.391602
2006-01-06,1412.780029,NaN,5939.99707,126000.0,38778.527344,NaN,9450.0,1665.060181,5671.862305,1484.561646,...,36300.035156,8286.917969,44000.0,17750.0,22377.132812,834.262634,5083.181641,3323.076904,4112.900391,2190.593994


In [89]:
kospi['^KS11'].isna().sum()

14

In [90]:
# 열을 검사하여 5열 연속 비어있는 열의 경우 -1로 변경
#향후 보간법 적용을 원활히 하기 위함
# 결측치를 -1로 변경하는 함수 정의
def replace_long_nan_streaks(series, threshold=5):
    count = 0
    for i in range(len(series)):
        if pd.isna(series[i]):
            count += 1
        else:
            if count >= threshold:
                series[i-count:i] = -1
            count = 0
    if count >= threshold:  # 끝에 도달했을 때 연속된 NaN이 threshold 이상인 경우
        series[-count:] = -1
    return series

# 각 열에 대해 결측치를 -1로 변경
for col in kospi.columns:
    kospi[col] = replace_long_nan_streaks(kospi[col])

In [91]:
kospi.isna().sum().sum()

7237

In [92]:
# 선형 보간법을 사용하여 NaN 값 채우기
kospi = kospi.interpolate(method='linear')

In [93]:
#다시 Nan으로 변경
kospi.replace(-1, np.nan, inplace=True)

In [95]:
kospi.head()

,^KS11,093050,000040,077970,003550,088790,072130,009070,000320,023960,...,005980,001500,005180,000990,011000,001570,009410,008040,002630,003220
date,,,,,,,,,,,,,,,,,,,,,
2006-01-02,1389.270020,NaN,5879.99707,126400.0,34390.574219,NaN,9000.0,1658.556030,5508.643555,1540.254395,...,NaN,8971.159180,39250.0,16400.0,16608.988281,862.071350,4607.680176,3023.076904,4103.958984,2208.189209
2006-01-03,1394.869995,NaN,6119.99707,135200.0,33512.984375,NaN,9000.0,1632.539429,5427.033691,1543.735229,...,36750.035156,8743.079102,38700.0,16200.0,16854.441406,866.706177,4554.102539,3038.461426,4247.016602,2202.324219
2006-01-04,1402.109985,NaN,5999.99707,123200.0,34829.367188,NaN,9420.0,1652.051880,5427.033691,1561.139282,...,36750.035156,8438.971680,44500.0,16100.0,19349.878906,857.436584,4688.046387,3292.307617,4077.135742,2202.324219
2006-01-05,1395.510010,NaN,5999.99707,124800.0,35761.808594,NaN,9350.0,1652.051880,5488.241211,1482.821289,...,36450.035156,8438.971680,44500.0,16800.0,19718.058594,829.627808,4554.102539,3292.307617,4050.312500,2199.391602
2006-01-06,1412.780029,NaN,5939.99707,126000.0,38778.527344,NaN,9450.0,1665.060181,5671.862305,1484.561646,...,36300.035156,8286.917969,44000.0,17750.0,22377.132812,834.262634,5083.181641,3323.076904,4112.900391,2190.593994


In [94]:
# Save the updated dataframe to a CSV file
kospi.to_csv('.\kospi\clean_kospi_stock_data.csv', index=True)